# 🎯 Calibration for LIDAR Grasp v13

## ค่าที่ต้อง Calibrate ทั้งหมด
| ค่า | หน่วย | วิธี |
|-----|------|------|
| PIXELS_PER_MM | px/mm | วัดวัตถุจริง vs pixel |
| HOMOGRAPHY_MATRIX | - | 4-point calibration |
| ROBOT_R_OFFSET | องศา | หมุน gripper ให้ตรง |
| Z_FLOOR | mm | หุ่นยนต์แตะพื้น |
| LIDAR_X_OFFSET | mm | offset X LIDAR→Gripper |
| LIDAR_Y_OFFSET | mm | offset Y LIDAR→Gripper |
| LIDAR_PHYSICAL_OFFSET | mm | ระยะ LIDAR ถึง gripper tip |
| LIDAR_CORRECTION | mm | แก้ error LIDAR |
| HEIGHT_CORRECTION_FACTOR | % | factor ปรับตามความสูง |
| GRIPPER_MAX_WIDTH_MM | mm | ความกว้าง gripper เปิดสุด |

## 1️⃣ Imports & Setup

In [ ]:
import cv2
import numpy as np
import time
import socket
import serial

# Hardware
ROBOT_IP = '192.168.1.6'
ESP32_PORT = 'COM9'
ESP32_BAUDRATE = 115200
CAMERA_ID = 2

print("✓ Setup complete")

---
# 📏 PART 1: PIXELS_PER_MM Calibration
---
## วิธีทำ:
1. วางวัตถุที่รู้ขนาดจริง (เช่น ไม้บรรทัด หรือกล่องที่วัดขนาด)
2. Run cell แล้วคลิก 2 จุด เพื่อวัดระยะ pixel
3. คำนวณ: `PIXELS_PER_MM = distance_pixels / distance_mm`

In [ ]:
# === PIXELS PER MM CALIBRATION ===
points = []

def click_ppm(event, x, y, flags, param):
    global points
    if event == cv2.EVENT_LBUTTONDOWN and len(points) < 2:
        points.append((x, y))
        print(f"Point {len(points)}: ({x}, {y})")

cap = cv2.VideoCapture(CAMERA_ID)
cv2.namedWindow('PPM Calibration')
cv2.setMouseCallback('PPM Calibration', click_ppm)

print("🔵 คลิก 2 จุด บนวัตถุที่รู้ขนาดจริง (เช่น ไม้บรรทัด)")
print("กด 'q' เมื่อคลิกครบ 2 จุด")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    
    display = frame.copy()
    for i, pt in enumerate(points):
        cv2.circle(display, pt, 5, (0, 0, 255), -1)
        cv2.putText(display, f"P{i+1}", (pt[0]+10, pt[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)
    
    if len(points) == 2:
        cv2.line(display, points[0], points[1], (0, 255, 0), 2)
        dist = np.sqrt((points[1][0]-points[0][0])**2 + (points[1][1]-points[0][1])**2)
        cv2.putText(display, f"{dist:.1f} px", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    
    cv2.imshow('PPM Calibration', display)
    if cv2.waitKey(1) & 0xFF == ord('q'): break

cap.release()
cv2.destroyAllWindows()

if len(points) == 2:
    pixel_dist = np.sqrt((points[1][0]-points[0][0])**2 + (points[1][1]-points[0][1])**2)
    print(f"\n📏 ระยะที่วัดได้: {pixel_dist:.1f} pixels")

In [ ]:
# === คำนวณ PIXELS_PER_MM ===
# ใส่ขนาดจริงของวัตถุที่วัด (mm)
ACTUAL_SIZE_MM = 50  # <-- แก้ตรงนี้! (เช่น 100mm ถ้าวัด 10cm)

PIXELS_PER_MM = pixel_dist / ACTUAL_SIZE_MM
print(f"\n✅ PIXELS_PER_MM = {PIXELS_PER_MM:.4f}")
print(f"\n📋 Copy ไปใช้: PIXELS_PER_MM = {PIXELS_PER_MM:.4f}")

---
# 📍 PART 2: HOMOGRAPHY_MATRIX Calibration (4-Point)
---
## วิธีทำ:
1. วางจุด marker 4 จุด บนโต๊ะทำงาน (เช่น กระดาษสีหรือ sticker)
2. ใช้ Robot บันทึกพิกัด (X, Y) ที่ marker แต่ละจุด
3. Run cell แล้วคลิกที่ marker ตามลำดับ A, B, C, D
4. ระบบจะคำนวณและ save Homography Matrix

In [ ]:
# === Robot Controller สำหรับ Calibration ===
class CalibRobot:
    def __init__(self, ip):
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.sock.settimeout(5)
        self.sock.connect((ip, 29999))
        self.send("ClearError()")
        time.sleep(0.5)
        self.send("EnableRobot()")
        time.sleep(4)
        self.send("User(1)")
        self.send("Tool(1)")
        self.send("SpeedFactor(30)")
        print("✅ Robot connected")
    
    def send(self, cmd):
        self.sock.send((cmd + "\n").encode())
        return self.sock.recv(1024).decode()
    
    def move(self, x, y, z, r=0):
        print(f"Moving to ({x}, {y}, {z}, {r})")
        return self.send(f"MovJ({x},{y},{z},{r})")

robot = CalibRobot(ROBOT_IP)
print("\n🤖 ใช้ Dobot Studio บันทึกพิกัด X,Y ที่ marker แต่ละจุด")

In [ ]:
# === ใส่ค่า Robot Coordinates X,Y ที่วัดได้ ===
# เรียงตามลำดับ A, B, C, D (ตามเข็มหรือทวนเข็มนาฬิกา)

ROBOT_COORDS = {
    'A': (300.0, 100.0),   # <-- แก้ค่าตรงนี้!
    'B': (300.0, -100.0),  # <-- แก้ค่าตรงนี้!
    'C': (400.0, -100.0),  # <-- แก้ค่าตรงนี้!
    'D': (400.0, 100.0),   # <-- แก้ค่าตรงนี้!
}

print("📍 Robot Coordinates:")
for name, coord in ROBOT_COORDS.items():
    print(f"   {name}: {coord}")

In [ ]:
# === คลิกเลือก Pixel Coordinates ===
pixel_coords = []
labels = ['A', 'B', 'C', 'D']

def click_homo(event, x, y, flags, param):
    global pixel_coords
    if event == cv2.EVENT_LBUTTONDOWN and len(pixel_coords) < 4:
        pixel_coords.append((x, y))
        print(f"{labels[len(pixel_coords)-1]}: pixel ({x}, {y})")

cap = cv2.VideoCapture(CAMERA_ID)
cv2.namedWindow('Homography')
cv2.setMouseCallback('Homography', click_homo)

print("🔵 คลิก marker ตามลำดับ: A → B → C → D")
print("กด 'q' เมื่อครบ 4 จุด")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    
    display = frame.copy()
    for i, pt in enumerate(pixel_coords):
        cv2.circle(display, pt, 8, (0, 0, 255), -1)
        cv2.putText(display, labels[i], (pt[0]+10, pt[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
    
    cv2.putText(display, f"Clicked: {len(pixel_coords)}/4", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
    cv2.imshow('Homography', display)
    if cv2.waitKey(1) & 0xFF == ord('q'): break

cap.release()
cv2.destroyAllWindows()

In [ ]:
# === คำนวณและ Save Homography Matrix ===
if len(pixel_coords) == 4:
    src = np.array(pixel_coords, dtype=np.float32)
    dst = np.array([ROBOT_COORDS['A'], ROBOT_COORDS['B'], ROBOT_COORDS['C'], ROBOT_COORDS['D']], dtype=np.float32)
    
    H, _ = cv2.findHomography(src, dst)
    np.save('homography_matrix.npy', H)
    
    print("\n✅ HOMOGRAPHY_MATRIX saved to 'homography_matrix.npy'")
    print(f"\nMatrix:\n{H}")
else:
    print("❌ ต้องคลิกครบ 4 จุด!")

---
# 🔄 PART 3: ROBOT_R_OFFSET Calibration
---
## วิธีทำ:
1. วาง object ยาว (เช่น ปากกา) ให้ขนานกับแนวแกน X ของ Robot
2. ให้ camera detect และ PCA คำนวณมุม
3. ปรับ ROBOT_R_OFFSET จนกว่า gripper จะหมุนขนานกับ object

In [ ]:
# === Test R Offset ===
# ค่าเริ่มต้น (ปรับจนกว่า gripper จะตรง)
TEST_R_OFFSET = -25.55  # <-- ปรับค่านี้!

# ถ้ากำหนด camera_angle = 0 (object ตรง)
# robot_r = ROBOT_R_OFFSET - camera_angle = ROBOT_R_OFFSET
# ถ้า gripper หมุนผิดทิศ, ให้ปรับ ROBOT_R_OFFSET

print(f"🔄 ทดสอบ: ROBOT_R_OFFSET = {TEST_R_OFFSET}")
print(f"   วาง object ขนานแกน X แล้วดูว่า gripper ตรงไหม")
print(f"   - ถ้าเบี้ยวซ้าย → เพิ่ม R_OFFSET")
print(f"   - ถ้าเบี้ยวขวา → ลด R_OFFSET")

---
# 📐 PART 4: Z_FLOOR Calibration
---
## วิธีทำ:
1. ใช้ Teach Mode ใน Dobot Studio
2. ขยับ gripper ลงแตะพื้นโต๊ะเบาๆ
3. อ่านค่า Z จาก Dobot Studio → นั่นคือ Z_FLOOR

In [ ]:
# === Z_FLOOR ===
# ค่าที่อ่านได้จาก Dobot Studio เมื่อ gripper แตะพื้น
Z_FLOOR = -64  # <-- แก้ค่าจากที่วัดได้!

print(f"✅ Z_FLOOR = {Z_FLOOR}")
print(f"\n📋 Copy: Z_FLOOR = {Z_FLOOR}")

---
# 📏 PART 5: LIDAR Offset Calibration
---
## ค่าที่ต้อง Calibrate:
1. **LIDAR_X_OFFSET** - ระยะ X จาก gripper center → LIDAR
2. **LIDAR_Y_OFFSET** - ระยะ Y จาก gripper center → LIDAR
3. **LIDAR_PHYSICAL_OFFSET** - ระยะแนวตั้ง LIDAR → gripper tip
4. **LIDAR_CORRECTION** - ค่าแก้ error LIDAR

In [16]:
# === LIDAR XY OFFSET CALIBRATION ===
# วิธี: ใช้หุ่นยนต์ชี้ไปที่จุด target แล้วเทียบกับ LIDAR

print("=" * 60)
print("📏 LIDAR XY OFFSET CALIBRATION")
print("=" * 60)

# ขั้นตอน:
# 1. วาง marker (กระดาษสีเล็กๆ) บนพื้นโต๊ะ
# 2. ใช้ Dobot Studio ขยับ GRIPPER CENTER ไปอยู่เหนือ marker พอดี
# 3. บันทึกค่า (X, Y) จาก Dobot Studio → นั่นคือ GRIPPER_POS
# 4. ขยับหุ่นยนต์ให้ LIDAR อยู่เหนือ marker เดิม
# 5. บันทึกค่า (X, Y) อีกครั้ง → นั่นคือ LIDAR_POS
# 6. คำนวณ Offset

print("""
📍 ขั้นตอนการ Calibrate:

1. วาง marker เล็กๆ (เช่น กระดาษสี 1x1 cm) บนพื้นโต๊ะ

2. ใช้ Dobot Studio (Teach Mode) ขยับหุ่นยนต์ให้:
   ↳ จุดกึ่งกลาง GRIPPER อยู่ตรงกับ marker พอดี
   ↳ บันทึกค่า X, Y

3. ใช้ Dobot Studio ขยับหุ่นยนต์ให้:
   ↳ จุด LIDAR (เซ็นเซอร์) อยู่ตรงกับ marker เดิมพอดี
   ↳ บันทึกค่า X, Y

4. คำนวณ:
   ↳ LIDAR_X_OFFSET = LIDAR_X - GRIPPER_X
   ↳ LIDAR_Y_OFFSET = LIDAR_Y - GRIPPER_Y
""")

# ใส่ค่าจาก Dobot Studio
GRIPPER_X = 350.0  # <-- แก้ค่าตรงนี้! (X เมื่อ gripper center อยู่เหนือ marker)
GRIPPER_Y = 0.0    # <-- แก้ค่าตรงนี้! (Y เมื่อ gripper center อยู่เหนือ marker)

LIDAR_X = 375.08   # <-- แก้ค่าตรงนี้! (X เมื่อ LIDAR อยู่เหนือ marker)
LIDAR_Y = 20.71    # <-- แก้ค่าตรงนี้! (Y เมื่อ LIDAR อยู่เหนือ marker)

# คำนวณ Offset
LIDAR_X_OFFSET = LIDAR_X - GRIPPER_X
LIDAR_Y_OFFSET = LIDAR_Y - GRIPPER_Y

print("=" * 60)
print("📊 ผลการ Calibrate:")
print("=" * 60)
print(f"   Gripper Center: ({GRIPPER_X}, {GRIPPER_Y})")
print(f"   LIDAR Position: ({LIDAR_X}, {LIDAR_Y})")
print()
print(f"✅ LIDAR_X_OFFSET = {LIDAR_X_OFFSET:.2f} mm")
print(f"   (+ = LIDAR อยู่ด้านหน้า gripper / - = อยู่ด้านหลัง)")
print()
print(f"✅ LIDAR_Y_OFFSET = {LIDAR_Y_OFFSET:.2f} mm")
print(f"   (+ = LIDAR อยู่ด้านซ้าย gripper / - = อยู่ด้านขวา)")
print()
print("=" * 60)
print("📋 Copy ไปใส่ใน v13 Notebook:")
print("=" * 60)
print(f"LIDAR_X_OFFSET = {LIDAR_X_OFFSET:.2f}  # mm")
print(f"LIDAR_Y_OFFSET = {LIDAR_Y_OFFSET:.2f}  # mm")

📏 LIDAR XY OFFSET CALIBRATION

📍 ขั้นตอนการ Calibrate:

1. วาง marker เล็กๆ (เช่น กระดาษสี 1x1 cm) บนพื้นโต๊ะ

2. ใช้ Dobot Studio (Teach Mode) ขยับหุ่นยนต์ให้:
   ↳ จุดกึ่งกลาง GRIPPER อยู่ตรงกับ marker พอดี
   ↳ บันทึกค่า X, Y

3. ใช้ Dobot Studio ขยับหุ่นยนต์ให้:
   ↳ จุด LIDAR (เซ็นเซอร์) อยู่ตรงกับ marker เดิมพอดี
   ↳ บันทึกค่า X, Y

4. คำนวณ:
   ↳ LIDAR_X_OFFSET = LIDAR_X - GRIPPER_X
   ↳ LIDAR_Y_OFFSET = LIDAR_Y - GRIPPER_Y

📊 ผลการ Calibrate:
   Gripper Center: (350.0, 0.0)
   LIDAR Position: (375.08, 20.71)

✅ LIDAR_X_OFFSET = 25.08 mm
   (+ = LIDAR อยู่ด้านหน้า gripper / - = อยู่ด้านหลัง)

✅ LIDAR_Y_OFFSET = 20.71 mm
   (+ = LIDAR อยู่ด้านซ้าย gripper / - = อยู่ด้านขวา)

📋 Copy ไปใส่ใน v13 Notebook:
LIDAR_X_OFFSET = 25.08  # mm
LIDAR_Y_OFFSET = 20.71  # mm


In [ ]:
# === 5.1: LIDAR XY Offset ===
# วัดด้วยไม้บรรทัด จาก gripper center ไป LIDAR

LIDAR_X_OFFSET = 25.08  # mm (+ = LIDAR อยู่ด้านหน้า gripper)
LIDAR_Y_OFFSET = 20.71  # mm (+ = LIDAR อยู่ด้านซ้าย gripper)

print(f"📏 LIDAR XY Offset:")
print(f"   X: {LIDAR_X_OFFSET} mm")
print(f"   Y: {LIDAR_Y_OFFSET} mm")

In [14]:
# === 5.2: LIDAR Physical Offset & Correction ===
# เชื่อมต่อ LIDAR ก่อน

class LidarTest:
    def __init__(self, port):
        self.serial = serial.Serial(port, 115200, timeout=2)
        time.sleep(2)
        print("✅ LIDAR connected")
    
    def read(self, samples=10):
        readings = []
        for _ in range(samples):
            self.serial.reset_input_buffer()
            self.serial.write(b'L\n')
            time.sleep(0.1)
            if self.serial.in_waiting:
                resp = self.serial.readline().decode().strip()
                if resp.startswith("LIDAR:") and "ERR" not in resp:
                    readings.append(int(resp.split(":")[1]))
        return int(np.median(readings)) if readings else None

lidar = LidarTest(ESP32_PORT)

✅ LIDAR connected


In [15]:
# === 5.3: วัด LIDAR Physical Offset ===
# 1. เอา gripper ลงแตะพื้น (Z = Z_FLOOR)
# 2. อ่านค่า LIDAR
# 3. LIDAR_PHYSICAL_OFFSET = ค่า LIDAR ที่อ่านได้

print("📏 ขั้นตอน:")
print("1. ใช้ Dobot Studio ขยับ gripper แตะพื้น (Z = Z_FLOOR)")
print("2. กด Enter เพื่ออ่านค่า LIDAR")
input("\nPress Enter when gripper is at floor level...")

lidar_at_floor = lidar.read(samples=20)
print(f"\n📏 LIDAR reading at floor: {lidar_at_floor} mm")
print(f"\n✅ LIDAR_PHYSICAL_OFFSET = {lidar_at_floor}")

📏 ขั้นตอน:
1. ใช้ Dobot Studio ขยับ gripper แตะพื้น (Z = Z_FLOOR)
2. กด Enter เพื่ออ่านค่า LIDAR

📏 LIDAR reading at floor: 30 mm

✅ LIDAR_PHYSICAL_OFFSET = 30


In [ ]:
# === 5.4: LIDAR Correction Test ===
# วางวัตถุที่รู้ความสูงจริง แล้วเทียบกับค่า LIDAR

Z_MEASURE = 120  # ความสูง Z ที่จะวัด LIDAR
KNOWN_HEIGHT_MM = 50  # ความสูงจริงของวัตถุ (เช่น กล่องสูง 50mm)

print(f"📏 วาง object สูง {KNOWN_HEIGHT_MM}mm ใต้ LIDAR")
print(f"   หุ่นยนต์อยู่ที่ Z = {Z_MEASURE}")
input("\nPress Enter when ready...")

lidar_reading = lidar.read(samples=20)
print(f"\n📏 LIDAR reading: {lidar_reading} mm")

# คำนวณ
# z_calculated = Z_MEASURE - lidar_reading + LIDAR_PHYSICAL_OFFSET
# ค่าที่คาดหวัง = Z_FLOOR + KNOWN_HEIGHT_MM
expected_z = Z_FLOOR + KNOWN_HEIGHT_MM
calculated_z = Z_MEASURE - lidar_reading + lidar_at_floor

LIDAR_CORRECTION = expected_z - calculated_z

print(f"\n📊 ผลลัพธ์:")
print(f"   Expected Z (grasp): {expected_z}")
print(f"   Calculated Z: {calculated_z}")
print(f"\n✅ LIDAR_CORRECTION = {LIDAR_CORRECTION:.1f}")

---
# 📈 PART 6: HEIGHT_CORRECTION_FACTOR Calibration
---
## วิธีทำ:
1. ทดสอบ pick วัตถุที่ความสูงต่างๆ
2. ถ้า gripper ลงลึกเกินไป → เพิ่ม factor
3. ถ้า gripper ไม่ถึง → ลด factor

In [ ]:
# === HEIGHT_CORRECTION_FACTOR ===
# ค่าแนะนำเริ่มต้น: 0.10 - 0.20 (10-20%)

HEIGHT_CORRECTION_FACTOR = 0.115  # <-- ปรับจากการทดสอบ

print(f"📈 HEIGHT_CORRECTION_FACTOR = {HEIGHT_CORRECTION_FACTOR}")
print(f"\nวิธีปรับ:")
print(f"   - gripper ลงลึกเกินไป → เพิ่ม factor (เช่น 0.12 → 0.15)")
print(f"   - gripper ไม่ถึง → ลด factor (เช่น 0.12 → 0.10)")

---
# 🦾 PART 7: Gripper Calibration
---
## วิธีทำ:
1. ส่งค่า angle ต่างๆ ไปที่ gripper
2. วัดความกว้าง gripper จริง (mm)
3. สร้าง lookup table สำหรับ interpolation

In [ ]:
# === Gripper Controller ===
class GripperCalib:
    def __init__(self, port):
        self.serial = serial.Serial(port, 115200, timeout=2)
        time.sleep(2)
        print("✅ Gripper connected")
    
    def set_angle(self, angle):
        self.serial.write(f'G{angle}\n'.encode())
        time.sleep(0.5)
        print(f"Set angle: {angle}°")

gripper = GripperCalib(ESP32_PORT)

In [ ]:
# === ทดสอบแต่ละมุม ===
# รันทีละบรรทัด แล้ววัดความกว้างจริง

test_angles = [22, 30, 40, 50, 60, 70, 80, 90, 96]

for angle in test_angles:
    gripper.set_angle(angle)
    width = input(f"Angle {angle}° → กว้างกี่ mm? ")
    print(f"   → {angle}°: {width}mm")

In [ ]:
# === Gripper Lookup Table ===
# ค่า Default (แก้ไขตามที่วัดได้)

CALIB_ANGLES = np.array([22, 30, 40, 50, 60, 70, 80, 90, 96])
CALIB_WIDTHS = np.array([54.0, 52.0, 48.0, 40.0, 32.0, 23.0, 12.0, 3.0, 0.0])

GRIPPER_MAX_WIDTH_MM = CALIB_WIDTHS[0]  # ความกว้างสูงสุด

print("✅ Gripper Lookup Table:")
for a, w in zip(CALIB_ANGLES, CALIB_WIDTHS):
    print(f"   {a}° → {w}mm")
print(f"\nGRIPPER_MAX_WIDTH_MM = {GRIPPER_MAX_WIDTH_MM}")

---
# ✅ SUMMARY: All Calibrated Values for V13
---

In [ ]:
# === รวมค่าทั้งหมด ===
print("="*60)
print("📋 CALIBRATED VALUES FOR V13")
print("="*60)

summary = f'''
# === Copy ไปใส่ใน v13 Notebook ===

# Camera Calibration
PIXELS_PER_MM = {PIXELS_PER_MM:.4f}

# Robot R Rotation
ROBOT_R_OFFSET = {TEST_R_OFFSET}

# Z Heights
Z_FLOOR = {Z_FLOOR}
Z_MEASURE = {Z_MEASURE}

# LIDAR Configuration
LIDAR_PHYSICAL_OFFSET = {lidar_at_floor}  # mm
LIDAR_CORRECTION = {LIDAR_CORRECTION:.0f}  # mm
LIDAR_X_OFFSET = {LIDAR_X_OFFSET}
LIDAR_Y_OFFSET = {LIDAR_Y_OFFSET}

# Height-based Correction
HEIGHT_CORRECTION_FACTOR = {HEIGHT_CORRECTION_FACTOR}

# Gripper
GRIPPER_MAX_WIDTH_MM = {GRIPPER_MAX_WIDTH_MM}
GRIPPER_OPEN_MARGIN_MM = 5
GRIPPER_GRIP_MARGIN_MM = 5

# Detection
MIN_OBJECT_AREA = 800
MAX_OBJECT_AREA = 50000
'''

print(summary)

In [ ]:
# === Save Summary to File ===
with open('calibration_values_v13.txt', 'w') as f:
    f.write(summary)
print("✅ Saved to 'calibration_values_v13.txt'")